In [1]:
from yacs.config import CfgNode as CN
import multiprocessing as mp
import os
import cv2
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle
from tensorflow.keras import layers
import random
from yolov4.tf import YOLOv4
from yolov4.model import yolov4
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk


2024-03-09 19:31:31.227008: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/devd/anaconda3/envs/devd-darty/lib/python3.7/site-packages/cv2/../../lib64:
2024-03-09 19:31:31.227031: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-09 19:31:32.163217: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2024-03-09 19:31:32.177706: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-09 19:31:32.177894: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVID

In [2]:
def make_model(
        yolo,
        activation0: str = "mish",
        activation1: str = "leaky",
        kernel_regularizer=tf.keras.regularizers.l2(0.0005),
):
    yolo._has_weights = False
    inputs = layers.Input([yolo.input_size[1], yolo.input_size[0], 3])
    if yolo.tiny:
        yolo.model = yolov4.YOLOv4Tiny(
            anchors=yolo.anchors,
            num_classes=len(yolo.classes),
            xyscales=yolo.xyscales,
            activation=activation1,
            kernel_regularizer=kernel_regularizer,
        )
    else:
        yolo.model = yolov4.YOLOv4(
            anchors=yolo.anchors,
            num_classes=len(yolo.classes),
            xyscales=yolo.xyscales,
            activation0=activation0,
            activation1=activation1,
            kernel_regularizer=kernel_regularizer,
        )
    yolo.model(inputs)

In [3]:
def build_model(cfg, classes='classes'):
    yolo = YOLOv4(tiny=cfg.model.tiny)
    yolo.classes = classes
    yolo.input_size = (cfg.model.input_size, cfg.model.input_size)
    yolo.batch_size = cfg.train.batch_size
    make_model(yolo)
    return yolo

In [4]:
cfg = CN(new_allowed=True)
cfg.merge_from_file(os.path.join('configs','devddarty.yaml'))
cfg.model.name = 'devdarty'

yolo = build_model(cfg)
yolo.load_weights(os.path.join('models', 'devddarty', 'weights'), cfg.model.weights_type)

2024-03-09 19:31:32.239348: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-09 19:31:32.247008: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 1999965000 Hz
2024-03-09 19:31:32.247566: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5b2bb7906950 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-03-09 19:31:32.247595: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-03-09 19:31:32.249822: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1257] Device interconnect StreamExecutor with strength 1 edge matrix:
2024-03-09 19:31:32.249869: I tensorflow/core/common_runtime/gpu/gpu_dev

In [5]:
BOARD_DICT = {
    0: '13', 1: '4', 2: '18', 3: '1', 4: '20', 5: '5', 6: '12', 7: '9', 8: '14', 9: '11',
    10: '8', 11: '16', 12: '7', 13: '19', 14: '3', 15: '17', 16: '2', 17: '15', 18: '10', 19: '6'
}


In [6]:
def bboxes_to_xy(bboxes, max_darts=3):
    xy = np.zeros((4 + max_darts, 3), dtype=np.float32)
    for cls in range(5):
        if cls == 0:
            dart_xys = bboxes[bboxes[:, 4] == 0, :2][:max_darts]
            xy[4:4 + len(dart_xys), :2] = dart_xys
        else:
            cal = bboxes[bboxes[:, 4] == cls, :2]
            if len(cal):
                xy[cls - 1, :2] = cal[0]
    xy[(xy[:, 0] > 0) & (xy[:, 1] > 0), -1] = 1
    if np.sum(xy[:4, -1]) == 4:
        return xy
    else:
        xy = est_cal_pts(xy)
    return xy


In [7]:
def board_radii(r_d, cfg):
    r_t = r_d * (cfg.board.r_treble / cfg.board.r_double)  # treble radius, in px
    r_ib = r_d * (cfg.board.r_inner_bull / cfg.board.r_double)  # inner bull radius, in px
    r_ob = r_d * (cfg.board.r_outer_bull / cfg.board.r_double) # outer bull radius, in px
    w_dt = cfg.board.w_double_treble * (r_d / cfg.board.r_double)  # width of double and treble
    return r_t, r_ob, r_ib, w_dt

In [8]:
def get_circle(xy):
    c = np.mean(xy[:4], axis=0)
    r = np.mean(np.linalg.norm(xy[:4] - c, axis=-1))
    return c, r


In [9]:
def transform(xy, img=None, angle=9, M=None):

    if xy.shape[-1] == 3:
        has_vis = True
        vis = xy[:, 2:]
        xy = xy[:, :2]
    else:
        has_vis = False

    if img is not None and np.mean(xy[:4]) < 1:
        h, w = img.shape[:2]
        xy *= [[w, h]]

    if M is None:
        c, r = get_circle(xy)  # not necessarily a circle
        # c is center of 4 calibration points, r is mean distance from center to calibration points

        src_pts = xy[:4].astype(np.float32)
        dst_pts = np.array([
            [c[0] - r * np.sin(np.deg2rad(angle)), c[1] - r * np.cos(np.deg2rad(angle))],
            [c[0] + r * np.sin(np.deg2rad(angle)), c[1] + r * np.cos(np.deg2rad(angle))],
            [c[0] - r * np.cos(np.deg2rad(angle)), c[1] + r * np.sin(np.deg2rad(angle))],
            [c[0] + r * np.cos(np.deg2rad(angle)), c[1] - r * np.sin(np.deg2rad(angle))]
        ]).astype(np.float32)
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)

    xyz = np.concatenate((xy, np.ones((xy.shape[0], 1))), axis=-1).astype(np.float32)
    xyz_dst = np.matmul(M, xyz.T).T
    xy_dst = xyz_dst[:, :2] / xyz_dst[:, 2:]

    if img is not None:
        img = cv2.warpPerspective(img.copy(), M, (img.shape[1], img.shape[0]))
        xy_dst /= [[w, h]]

    if has_vis:
        xy_dst = np.concatenate([xy_dst, vis], axis=-1)

    return xy_dst, img, M

In [10]:
def get_dart_scores(xy, cfg, numeric=False):
    valid_cal_pts = xy[:4][(xy[:4, 0] > 0) & (xy[:4, 1] > 0)]
    if xy.shape[0] <= 4 or valid_cal_pts.shape[0] < 4:  # missing calibration point
        return []
    xy, _, _ = transform(xy.copy(), angle=0)
    c, r_d = get_circle(xy)
    r_t, r_ob, r_ib, w_dt = board_radii(r_d, cfg)
    xy -= c
    angles = np.arctan2(-xy[4:, 1], xy[4:, 0]) / np.pi * 180
    angles = [a + 360 if a < 0 else a for a in angles]  # map to 0-360
    distances = np.linalg.norm(xy[4:], axis=-1)
    scores = []
    for angle, dist in zip(angles, distances):
        if dist > r_d:
            scores.append('0')
        elif dist <= r_ib:
            scores.append('DB')
        elif dist <= r_ob:
            scores.append('B')
        else:
            number = BOARD_DICT[int(angle / 18)]
            if dist <= r_d and dist > r_d - w_dt:
                scores.append('D' + number)
            elif dist <= r_t and dist > r_t - w_dt:
                scores.append('T' + number)
            else:
                scores.append(number)
    if numeric:
        for i, s in enumerate(scores):
            if 'B' in s:
                if 'D' in s:
                    scores[i] = 50
                else:
                    scores[i] = 25
            else:
                if 'D' in s or 'T' in s:
                    scores[i] = int(s[1:])
                    scores[i] = scores[i] * 2 if 'D' in s else scores[i] * 3
                else:
                    scores[i] = int(s)
    return scores

In [11]:
def draw(img, xy, cfg, circles, score, color=(255, 255, 0)):
    xy = np.array(xy)
    if xy.shape[0] > 7:
        xy = xy.reshape((-1, 2))
    if np.mean(xy) < 1:
        h, w = img.shape[:2]
        xy[:, 0] *= w
        xy[:, 1] *= h
    if xy.shape[0] >= 4 and circles:
        img = draw_circles(img, xy, cfg)
    if xy.shape[0] > 4 and score:
        scores = get_dart_scores(xy, cfg)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2
    line_type = 1
    for i, [x, y] in enumerate(xy):
        if i < 4:
            c = (0, 255, 0)  # green
        else:
            c = color  # cyan
        x = int(round(x))
        y = int(round(y))
        if i >= 4:
            cv2.circle(img, (x, y), 1, c, 1)
            if score:
                txt = str(scores[i - 4])
            else:
                txt = str(i + 1)
            cv2.putText(img, txt, (x + 8, y), font,
                    font_scale, c, line_type)
        else:
            cv2.circle(img, (x, y), 1, c, 1)
            cv2.putText(img, str(i + 1), (x + 8, y), font,
                        font_scale, c, line_type)
    return img

In [12]:

def predictimage(
        yolo,
        cfg,
        img,
        max_darts=3):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    preds = np.zeros((1, 4 + max_darts, 3))
    bboxes = yolo.predict(img)
    preds[0] = bboxes_to_xy(bboxes, max_darts)
    print(preds)

    xy = preds[0]
    xy = xy[xy[:, -1] == 1]

    img = draw(cv2.cvtColor(img, cv2.COLOR_RGB2BGR), xy[:, :2], cfg, circles=False, score=True)
    return img
    

In [13]:
def perform_prediction():
    # Get the selected image file
    file = filedialog.askopenfile()

    if file:
        try:
            # Read the image using OpenCV
            img = cv2.imread(file.name)

           
            # Perform prediction using your predict function
            prediction_result = predictimage(yolo, cfg, img)
            img2 = Image.fromarray(prediction_result)
            img2 = ImageTk.PhotoImage(img2)
            
            result_label.config(image=img2)
            result_label.image = img2
            

            # Display the prediction result (for example, you can use a label or messagebox)

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred during prediction: {str(e)}")
    else:
        messagebox.showwarning("Warning", "No image selected.")


In [14]:

root = tk.Tk()
root.title("DevD Darty")
root.configure(bg="#f0f0f0")  # Set background color

# Custom color scheme
button_bg_color = "#4CAF50"  # Green
button_fg_color = "white"
button_hover_bg_color = "#45a049"  # Darker green

# Create a frame to hold the image
image_frame = tk.Frame(root)
image_frame.pack(padx=10, pady=10)

# Create a label to display the image
image_label = tk.Label(image_frame)
image_label.pack()

result_label = tk.Label(root, text="Prediction Result: ")
result_label.pack(pady=10)


# Create a button to select an image
select_image_button = tk.Button(root, text="Select Image", command=perform_prediction, bg=button_bg_color, fg=button_fg_color, activebackground=button_hover_bg_color, activeforeground=button_fg_color)
select_image_button.pack(pady=10)

# Run the application
root.mainloop()

[[[0.44105053 0.12769929 1.        ]
  [0.55886507 0.8723613  1.        ]
  [0.1276872  0.55890363 1.        ]
  [0.87228882 0.44085234 1.        ]
  [0.5538885  0.21592207 1.        ]
  [0.57373273 0.23035522 1.        ]
  [0.57560354 0.21153459 1.        ]]]
